In [ ]:
import os
import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import shapely
from shapely.geometry import (
    LineString,
    MultiLineString,
    MultiPoint,
    MultiPolygon,
    Point,
    Polygon,
)

import flopy
import flopy.discretization as fgrid
import flopy.plot as fplot
from flopy.utils import GridIntersect

print(sys.version)
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(mpl.__version__))
print("flopy version: {}".format(flopy.__version__))
print("shapely version: {}".format(shapely.__version__))

In [ ]:
import helpers
import conda_scripts
import conda_scripts.load_datasets as lsd
workspace = lsd.model_info.get_mod('son', True)


ml = conda_scripts.sv_budget.load_sv_model.get_model(workspace)

In [ ]:
z= helpers.get_zones(ml)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (6,8), dpi = 250)
mm= conda_scripts.make_map.make_map('Zones')
ax = mm.plotloc(fig, locname = 'SON_MOD')
z.dissolve('name').reset_index().plot('name',  ax = ax, alpha = .5)
z.dissolve('name').reset_index().exterior.plot( ax = ax, alpha = .5)
conda_scripts.plot_help.label_poly(z.dissolve('name').reset_index(),ax, column = 'name')

plt.savefig('zones.png', dpi = 250, bbox_inches = 'tight')

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
gdf = z.dissolve('name').reset_index()
# Assume gdf is your GeoDataFrame containing polygons and multipolygons

# Tolerance value for considering two points as 'shared'
tolerance = 1e-6

# Function to extract vertices from Polygon or MultiPolygon
def extract_vertices(geometry):
    if isinstance(geometry, Polygon):
        return list(geometry.exterior.coords)
    elif isinstance(geometry, MultiPolygon):
        # Loop through each polygon in the MultiPolygon and get the exterior coordinates
        vertices = []
        for poly in geometry.geoms:
            vertices.extend(list(poly.exterior.coords))
        return vertices
    else:
        return []  # Return an empty list if geometry is neither Polygon nor MultiPolygon

# Step 1: Extract the coordinates (vertices) of each polygon/multipolygon
gdf['vertices'] = gdf.geometry.apply(extract_vertices)

# Step 2: Find shared coordinates (vertices) with a tolerance
shared_coords = []
for i, row1 in gdf.query("name=='Highlands'").iterrows():
    for j, row2 in gdf.iterrows():
        if i != j:  # Avoid comparing the same polygon
            # Check if any vertex of row1 is within tolerance of row2's geometry
            for coord in row1['vertices']:
                point = Point(coord)
                # Check distance instead of contains, with a tolerance
                if row2.geometry.distance(point) < tolerance:
                    shared_coords.append(coord)

# Step 3: Create Point geometries from the shared coordinates
shared_points = [Point(coord) for coord in shared_coords]

mp = MultiPoint(
    points=shared_points
)
mp = mp.buffer(1000)


# Step 4: Create a new GeoDataFrame for the shared points
gdf_shared_points = gpd.GeoDataFrame(geometry=shared_points)

# # The shared_coords list now contains the coordinates that are shared across row
# shared_coords = np.array(shared_coords)

In [ ]:
from flopy.utils import GridIntersect

In [ ]:
mg = GridIntersect(ml.modelgrid, method="vertex")

In [ ]:
import pandas as pd

In [ ]:
ix = mg.intersect(mp)


In [ ]:
df = pd.DataFrame(list(ix['cellids']), columns=['col1', 'col2'])

fig = plt.figure(figsize = (6,8), dpi = 250)
mm= conda_scripts.make_map.make_map('Drain Cells')
ax = mm.plotloc(fig, locname = 'SON_MOD')

Map = flopy.plot.PlotMapView(ml,ax = ax)
Map.plot_grid(lw = .1)
Map.plot_ibound(ml.bas6.ibound.array[0])

array = np.zeros((ml.nrow, ml.ncol))
array[df.col1, df.col2] = 1
np.savetxt(os.path.join(workspace,'model_arrays', 'drain_locs.csv' ), array, delimiter = ',', fmt ='%d' )
np.savetxt(os.path.join('local_copy','model_arrays', 'drain_locs.csv' ), array, delimiter = ',', fmt ='%d' )
np.savetxt(os.path.join('pestaws_whiz','model_arrays', 'drain_locs.csv' ), array, delimiter = ',', fmt ='%d' )

array = np.ma.array(array, mask = array ==0)


Map.plot_array(array)

plt.savefig("Drain_cells.png", dpi =250, bbox_inches = 'tight')